# Database initialization

In [ ]:
# todo: create a script to build the logical model in the chosen NOSQL system

In [24]:
# Connection to Neo4j
from neo4j import GraphDatabase

URI = "bolt://127.0.0.1:7687"
USER = "neo4j"
PASSWORD = "neo4jproject123"

driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD), database="test")  # <-- Qui specificare il database

In [25]:
def run_query(query, params=None):
    with driver.session() as session:
        return session.run(query, params or {})

In [26]:
query = """
MATCH (n) DETACH DELETE n
"""
run_query(query)
print("Database cleared.")


Database cleared.


In [27]:
#Costrains
constraints = [
    "CREATE CONSTRAINT customer_id IF NOT EXISTS FOR (c:Customer) REQUIRE c.id IS UNIQUE",
    "CREATE CONSTRAINT terminal_id IF NOT EXISTS FOR (t:Terminal) REQUIRE t.id IS UNIQUE",
    "CREATE CONSTRAINT transaction_id IF NOT EXISTS FOR (tr:Transaction) REQUIRE tr.id IS UNIQUE"
]

for c in constraints:
    run_query(c)

print("Constraints created.")


Constraints created.


In [28]:
#Nodes
def create_customer(customer):
    query = """
    MERGE (c:Customer {id: $id})
    SET c.lat = $lat,
        c.lon = $lon,
        c.amountMean = $amountMean,
        c.amountStd = $amountStd,
        c.meanDailyTransactions = $meanDailyTransactions
    """
    run_query(query, customer)

def create_terminal(terminal):
    query = """
    MERGE (t:Terminal {id: $id})
    SET t.lat = $lat,
        t.lon = $lon
    """
    run_query(query, terminal)

def create_transaction(transaction):
    query = """
    MERGE (tr:Transaction {id: $id})
    SET tr.dateTime = datetime($dateTime),
        tr.amount = $amount,
        tr.isFraud = $isFraud
    """
    run_query(query, transaction)


In [ ]:
#Relationships
def link_customer_terminals(customer_id, terminal_list):
    query = """
    MATCH (c:Customer {id: $customer_id})
    UNWIND $terminal_list AS t_id
    MATCH (t:Terminal {id: t_id})
    MERGE (c)-[:USE]->(t)
    """
    run_query(query, {"customer_id": customer_id, "terminal_list": terminal_list})


def link_customer_transaction(customer_id, transaction_id):
    query = """
    MATCH (c:Customer {id: $customer_id})
    MATCH (tr:Transaction {id: $transaction_id})
    MERGE (c)-[:PERFORM]->(tr)
    """
    run_query(query, {"customer_id": customer_id, "transaction_id": transaction_id})

def link_terminal_transaction(terminal_id, transaction_id):
    query = """
    MATCH (t:Terminal {id: $terminal_id})
    MATCH (tr:Transaction {id: $transaction_id})
    MERGE (t)-[:ON]->(tr)
    """
    run_query(query, {"terminal_id": terminal_id, "transaction_id": transaction_id})


# DB POPULATION

In [39]:
import pandas as pd
customers = pd.read_csv("../data/dataset_1_customers.csv")
terminals = pd.read_csv("../data/dataset_1_terminals.csv")
transactions = pd.read_csv("../data/dataset_1.csv")


In [41]:
customers

,CUSTOMER_ID,x_customer_id,y_customer_id,mean_amount,std_amount,mean_nb_tx_per_day,available_terminals,nb_terminals
0,0,54.881350,71.518937,62.262521,31.131260,2.179533,"[29, 87, 144]",3
1,1,42.365480,64.589411,46.570785,23.285393,3.567092,"[5, 160]",2
2,2,96.366276,38.344152,80.213879,40.106939,2.115580,[],0
3,3,56.804456,92.559664,11.748426,5.874213,0.348517,"[65, 94, 113]",3
4,4,2.021840,83.261985,78.924891,39.462446,3.480049,[],0
...,...,...,...,...,...,...,...,...
995,995,43.216661,36.225882,58.023111,29.011555,0.305376,[],0
996,996,4.034602,51.110309,7.707631,3.853816,0.238208,"[32, 83, 194]",3
997,997,37.634146,6.177907,41.619615,20.809807,0.967002,"[147, 148, 185]",3
998,998,18.777030,40.467983,16.390871,8.195436,1.398557,[138],1


In [42]:
for _, row in customers.iterrows():
    customer = {
        "id": row["CUSTOMER_ID"],
        "lat": float(row["x_customer_id"]),
        "lon": float(row["y_customer_id"]),
        "amountMean": float(row["mean_amount"]),
        "amountStd": float(row["std_amount"]),
        "meanDailyTransactions": float(row["mean_nb_tx_per_day"])
    }
    create_customer(customer)


In [43]:
for _, row in terminals.iterrows():
    terminal = {
        "id": row["TERMINAL_ID"],
        "lat": float(row["x_terminal_id"]),
        "lon": float(row["y_terminal_id"])
    }
    create_terminal(terminal)

In [45]:
for _, row in transactions.iterrows():
    dateTime = row["TX_DATETIME"].replace(" ", "T")
    transaction = {
        "id": row["TRANSACTION_ID"],
        "dateTime": dateTime,
        "amount": float(row["TX_AMOUNT"]),
        "isFraud": bool(row["TX_FRAUD"])
    }
    create_transaction(transaction)


KeyboardInterrupt: 

## Relationships

In [ ]:
import ast

customers["available_terminals"] = customers["available_terminals"].apply(ast.literal_eval)

In [ ]:
for _, row in customers.iterrows():
    link_customer_terminals(row["id"], row["available_terminals"])


In [ ]:
for _, row in transactions.iterrows():
    link_customer_transaction(row["CUSTOMER_ID"], row["TRANSACTION_ID"])


In [ ]:
for _, row in transactions.iterrows():
    link_terminal_transaction(row["TERMINAL_ID"], row["TRANSACTION_ID"])

# End of connection

In [ ]:
driver.close()
print("Connection closed.")
